SimPy is a discrete-event simulation library. The behavior of active components (like vehicles, customers or messages) is modeled with processes. All processes live in an environment. They interact with the environment and with each other via events.

Processes are described by simple Python generators. You can call them process function or process method, depending on whether it is a normal function or method of a class. During their lifetime, they create events and yield them in order to wait for them to occur.

When a process yields an event, the process gets suspended. SimPy resumes the process, when the event occurs (we say that the event is processed). Multiple processes can wait for the same event. SimPy resumes them in the same order in which they yielded that event.

An important event type is the Timeout. Events of this type occur (are processed) after a certain amount of (simulated) time has passed. They allow a process to sleep (or hold its state) for the given time. A Timeout and all other events can be created by calling the appropriate method of the Environment that the process lives in (Environment.timeout() for example).

- ENVIRONMENT: è`il luogo dove avviene la simulazione, tutti gli eventi vengono gestiti da questo environment che fa accadere le attivià secondo l'ordine e la disponibilità corrente.
- GENERATOR: Un generatore è una funzione che non restituisce subito un risultato, ma lo restituisce a pezzi, nel tempo, usando yield.


# Esempio di generatore
In questo esempio la funzione "car" è di fatto un generatore, infatti si può notare come non contiene nessuna return ma solamente una yield.\
Yield: permette al processo corrente di arrestarsi in attesa che un altro evento accada, nel caso sottostante facciamo:

`yield env.timeout(parking_duration)`

Per dire che sospendiamo il processo fino a che il timeout non termina. Yield può essere usato anche in altri contesti, come per esempio sospendere il processo finché una richiesta di risorsa non è soddisfatta:

`with bus_seats.request() as req:
    yield req  # aspetta che ci sia posto`

 oppure

`yield fuel.get(5)   # prendi 5 litri di carburante`\
`yield fuel.put(10)  # aggiungi 10 litri`

La funzione è definita furbamente con un ciclo infinito, in modo tale da poter andare avanti indefinitivamente nel tempo, cosa che noi vogliamo quando simuliamo, dato che il tempo della simulazione viene definito quando avviamo il processo di simulazione.

In [1]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [3]:
import simpy
env = simpy.Environment()
env.process(car(env))

env.run(until=15)

Start parking at 0
Start parking at 0
Start driving at 5
Start driving at 5
Start parking at 7
Start parking at 7
Start driving at 12
Start driving at 12
Start parking at 14
Start parking at 14


# Classe del Processo

Per modellare problemi complessi, è spesso meglio creare la classe degli oggetti che stiamo modellando, questo perché le classi permettono di associare a quell'oggetto uno stato interno persistente.\
Immagina di voler fare un'analisi su tempi di attesa medi di clienti, ogni cliente potrebbe essere gestito come un oggetto a sè stante, con un tempo di attesa proprio.

### Interrompere un altro processo

Possiamo anche interrompere altri processi mentre stanno eseguendo delle attività (sono in stato di timeout).\
Per esempio possiamo interrompere l'attività dell'auto di prima mentre sta eseguendo la ricarica della batteria.

Quando utilizziamo yield, il processo si sospende e l’ambiente lo tiene in attesa finché non "scade" quel timeout.
Durante quel tempo è sospeso, e quindi:
- può essere interrotto (proc.interrupt())
- può essere riattivato da eventi esterni

Quando invece il processo non sta facendo yield, ma è "attivo" (cioè tra due yield), non può essere interrotto.

In [30]:
class Car(object):
    def __init__(self, env, name):
        self.env = env
        self.name = name
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print(f'{self.name} Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print(f'{self.name} Interrupted while charging at %d' % self.env.now)
                # If we are interrupted, we can skip the charge process
                continue

            # The charge process has finished and
            # we can start driving again.
            print(f'{self.name} Start driving at %d' % self.env.now)
            trip_duration = 2
            try:
                yield self.env.timeout(trip_duration)
            except simpy.Interrupt:
                print(f'{self.name} Interrupted while driving at %d' % self.env.now)
                # If we are interrupted, we can skip the trip process
                continue
    def charge(self, duration):
        yield self.env.timeout(duration)

In [22]:
env = simpy.Environment()
cars = [Car(env, f"Auto {i}") for i in range(3)]
env.run(until=15)

Auto 0 Start parking and charging at 0
Auto 1 Start parking and charging at 0
Auto 2 Start parking and charging at 0
Auto 0 Start driving at 5
Auto 1 Start driving at 5
Auto 2 Start driving at 5
Auto 0 Start parking and charging at 7
Auto 1 Start parking and charging at 7
Auto 2 Start parking and charging at 7
Auto 0 Start driving at 12
Auto 1 Start driving at 12
Auto 2 Start driving at 12
Auto 0 Start parking and charging at 14
Auto 1 Start parking and charging at 14
Auto 2 Start parking and charging at 14


In [23]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [31]:
env = simpy.Environment()
cars = [Car(env, f"Auto {i}") for i in range(3)]
env.process(driver(env, cars[0]))
env.run(until=15)

Auto 0 Start parking and charging at 0
Auto 1 Start parking and charging at 0
Auto 2 Start parking and charging at 0
Auto 0 Interrupted while charging at 3
Auto 0 Start parking and charging at 3
Auto 1 Start driving at 5
Auto 2 Start driving at 5
Auto 1 Start parking and charging at 7
Auto 2 Start parking and charging at 7
Auto 0 Start driving at 8
Auto 0 Start parking and charging at 10
Auto 1 Start driving at 12
Auto 2 Start driving at 12
Auto 1 Start parking and charging at 14
Auto 2 Start parking and charging at 14


# Risorse

Vorremmo poter modellare il fatto che uno o più oggetti debbano accedere a delle risorse (per esempio i posti dell'autobus) e quindi debbano richiedere quella specifica risorsa.\
Simpy permette di modellare una risorsa utilizzando 3 tipi di risorse:
- Resource: è il tipo principale di risorsa richiesta dai processi, Ogni processo richiede (request) e rilascia (release) una risorsa.
- Container: Per risorse quantitative, tipo acqua, carburante, soldi, passeggeri (in numero totale). Supporta .put(amount) e .get(amount). Non tiene traccia di chi prene una risorsa al contrario di Resource.
- Store: Per oggetti individuali e distinguibili da usare quando vuoi immagazzinare oggetti veri e propri (es. pacchi, messaggi, persone). Supporta .put(item) e .get() (oppure .get(filter_func))

In [ ]:
bcs = simpy.Resource(env, capacity=2)

# Callbacks

In [32]:
def my_callback(event):
    print('Called back from', event)

env = simpy.Environment()
event = env.event()
event.callbacks.append(my_callback)
event.callbacks

[<function __main__.my_callback(event)>]